In [ ]:
from pyiron import Project
import numpy as np
import matplotlib.pylab as plt
import json
from pandas import DataFrame
import os
%config InlineBackend.figure_format = 'retina'

In [ ]:
pr = Project('SNAP')

In [ ]:
lmp = pr.create.job.Lammps('lmp_Ta')
lmp.structure = pr.create.structure.bulk('Ta', cubic=True).repeat(4)
lmp.calc_md(temperature=300, n_ionic_steps=10000)
lmp.run()

In [ ]:
from pyiron_atomistics.atomistics.structure.atoms import pyiron_to_ase
from ase.calculators.singlepoint import SinglePointCalculator
import ase
def get_ase_structure(job, frame=-1):
    structure = pyiron_to_ase(job.get_structure(frame))
    args = {}
    try:
        args['forces'] = job['output/generic/forces'][frame]
        args['stress'] = -job['output/generic/pressures'][frame].flatten()[[0, 4, 8, 5, 2, 1]] * 1e4
        args['energy'] = job['output/generic/energy_pot'][frame]
    except:
        pass
    calc = SinglePointCalculator(structure, **args)
    structure.set_calculator(calc)
    return structure

In [ ]:
structures = [get_ase_structure(lmp, frame) for frame in np.arange(len(lmp.output.energy_pot))]

In [ ]:
ase.io.write('XYZ/test.xyz', structures)

In [ ]:
%cd /home/jovyan/testing
from fitsnap3lib.parallel_tools import ParallelTools
from fitsnap3lib.io.input import Config
pt = ParallelTools()
pt.check_fitsnap_exist=False
config = Config(arguments_lst=["./Ta-example.in", "--overwrite"])

# create a fitsnap object

from fitsnap3lib.fitsnap import FitSnap
snap = FitSnap()

In [ ]:
snap.scrape_configs()
snap.process_configs()
snap.perform_fit()

In [ ]:
lmp_ref = pr.load('lmp_Ta')

In [ ]:
lmp = pr.create.job.Lammps('lmp', delete_existing_job=True)
lmp.structure = lmp_ref.structure.copy()
# lmp.potential = lmp.list_potentials()[0]

path = os.path.abspath('.')
config = [
    'pair_style hybrid/overlay zbl 4.0 4.8 snap\n',
    'pair_coeff * * zbl 73 73\n',
    'pair_coeff * * snap {0}/Ta_pot.snapcoeff {0}/Ta_pot.snapparam Ta\n'.format(path)
]

potential = DataFrame.from_dict({
    'Config': [config],
    'Filename': [[]],
    'Name': ['Snap'],
    'Model': ['Snap'],
    'Species': [['Ta']],
    'Citations': [[]]
})

lmp.potential = potential

with lmp.interactive_open() as ll:
    for xx in lmp_ref.output.positions:
        ll.structure.positions = xx
        ll.run()

In [ ]:
plt.scatter(lmp_ref.output.energy_pot, lmp.output.energy_pot)